[Reference](https://medium.com/@DeveloperGlance/fastapi-endpoint-filters-9b21f6dade95)

In [2]:
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.8 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 


In [7]:
from typing import List

from fastapi import Depends, FastAPI
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.orm import Session, sessionmaker, declarative_base

# setup database
SQLALCHEMY_DATABASE_URL = "sqlite:///./sql_app.db"
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()
app = FastAPI()


# database definition
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_active = Column(Boolean, default=True)


Base.metadata.create_all(bind=engine)

# database related methods
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


def create_user(db: Session, email: str, password: str, is_active: bool = True):
    fake_hashed_password = password + "notreallyhashed"
    db_user = User(
        email=email, hashed_password=fake_hashed_password, is_active=is_active
    )
    db.add(db_user)


def get_users(db: Session) -> List[User]:
    return db.query(User).all()


# init database if needed
db = next(get_db())
if len(get_users(db)) == 0:
    create_user(db, "active1@example.com", "12345", True)
    create_user(db, "inactive1@example.com", "12345", False)
    create_user(db, "active2@example.com", "12345", True)
    create_user(db, "active3@example.com", "12345", True)


class SchemaUser(BaseModel):
    email: str
    id: int
    is_active: bool

    class Config:
        orm_mode = True


@app.get("/users/", response_model=List[SchemaUser])
def read_users(db: Session = Depends(get_db)):
    return list(map(SchemaUser.from_orm, get_users(db)))

In [6]:
from dataclasses import dataclass, asdict, is_dataclass
from typing import List, Optional

from fastapi import Depends, FastAPI, Query
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.orm import Session, sessionmaker, declarative_base

# setup database
SQLALCHEMY_DATABASE_URL = "sqlite:///./sql_app.db"
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()
app = FastAPI()


# database definition
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_active = Column(Boolean, default=True)
    is_superuser = Column(Boolean, default=False)


Base.metadata.create_all(bind=engine)

# database related methods
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


def create_user(
    db: Session,
    email: str,
    password: str,
    is_active: bool = True,
    is_superuser: bool = False,
):
    fake_hashed_password = password + "notreallyhashed"
    db_user = User(
        email=email,
        hashed_password=fake_hashed_password,
        is_active=is_active,
        is_superuser=is_superuser,
    )
    db.add(db_user)
    db.commit()


@dataclass
class UserFilters:
    is_active: Optional[bool] = None
    is_superuser: Optional[bool] = None

    def __init__(
        self,
        is_active: Optional[bool] = Query(
            None, description="Filter active/inactive users"
        ),
        is_superuser: Optional[bool] = Query(
            None, description="Filter normal users/superusers"
        ),
    ):
        self.is_superuser = is_superuser
        self.is_active = is_active


def get_multi(
    db: Session,
    model,
    filters=None,
) -> List:
    query = db.query(model)
    # we can make as many modification as we want to the
    # query before applying the generic filters and even exclude filters from the apply_filters function
    query = apply_filters(model, query, filters, )
    return query.all()


def apply_filters(model, query, filters=None, exclude: List[str] = ()):
    if filters is not None:
        if not is_dataclass(filters) or isinstance(filters, type):
            raise ValueError(f"Filters argument must be a dataclass")
        for filter_name, filter_value in asdict(filters).items():
            if filter_name not in exclude and filter_value is not None:
                model_attribute = getattr(model, filter_name, None)
                if model_attribute is not None:
                    query = query.filter(model_attribute == filter_value)
    return query


# init database if needed
db = next(get_db())
if len(get_multi(db, User)) == 0:
    create_user(db, "active1@example.com", "12345", True, True)
    create_user(db, "inactive1@example.com", "12345", False, False)
    create_user(db, "active2@example.com", "12345", True, False)
    create_user(db, "active3@example.com", "12345", True, False)


class SchemaUser(BaseModel):
    email: str
    id: int
    is_active: bool
    is_superuser: bool

    class Config:
        orm_mode = True


@app.get("/users/", response_model=List[SchemaUser])
def read_users(
    filters: UserFilters = Depends(),
    db: Session = Depends(get_db),
):
    return list(map(SchemaUser.from_orm, get_multi(db, User, filters)))